# Session 1 Hands-on Lab: Palawan Land Cover Classification with Random Forest
# INSTRUCTOR VERSION

**Duration:** 90 minutes (1.5 hours)

**Instructor:** CopPhil Advanced Training

---

## 📚 Instructor Notes

### Session Structure (90 minutes)

- **A-B: Setup & Study Area** (15 min) - Ensure all participants authenticate successfully
- **C: Data Acquisition** (20 min) - Emphasize cloud masking importance
- **D: Spectral Indices** (15 min) - Connect to physical properties
- **E: Feature Stack** (10 min) - Quick transition
- **F: Training Data** (20 min) - CRITICAL - stress data quality
- **G: RF Training** (20 min) - Explain hyperparameters clearly
- **H: Classification** (15 min) - Visual interpretation exercise
- **I: Accuracy Assessment** (25 min) - Deep dive into confusion matrix
- **J: Area Statistics** (10 min) - Connect to policy/management
- **K: Export** (10 min) - Quick demo
- **L: Advanced Exercises** (as time permits)

### Key Teaching Points

1. **Data quality > Model complexity**: Emphasize throughout
2. **Explainability matters**: Feature importance, visual inspection
3. **Context is critical**: Connect to Philippine NRM challenges
4. **Reproducibility**: Stress the importance of random seeds, documentation

### Common Participant Questions

**Q: Why Random Forest instead of deep learning?**
A: RF is interpretable, requires less training data, computationally efficient, and performs excellently for this problem. Deep learning benefits emerge with larger datasets and more complex features.

**Q: How much training data is enough?**
A: Rule of thumb: 50-100 pixels per class minimum. More for heterogeneous classes. Quality > quantity.

**Q: Why is accuracy assessment necessary if classification looks good?**
A: Visual inspection is subjective and biased. Quantitative metrics provide objective, reproducible performance measures required for scientific rigor and operational deployment.

**Q: Can we use this for change detection?**
A: Yes! Classify two time periods and compare. But be careful about seasonal effects, sensor calibration, and atmospheric conditions.

### Potential Issues

1. **Authentication problems**: Have backup authentication instructions ready
2. **Slow computation**: Remind participants GEE is server-side (their laptop speed doesn't matter)
3. **Memory errors**: Pre-prepared solutions with tileScale adjustments
4. **No images found**: Have backup date ranges ready

---

## Learning Objectives

By the end of this hands-on lab, participants will be able to:

1. **Set up and authenticate** Google Earth Engine (GEE) in Google Colab
2. **Acquire and preprocess** Sentinel-2 satellite imagery for a Philippine study area
3. **Calculate spectral indices** (NDVI, NDWI, NDBI, EVI) for land cover discrimination
4. **Create training datasets** by defining land cover classes and sampling spectral signatures
5. **Train a Random Forest classifier** using GEE's machine learning capabilities
6. **Perform land cover classification** on Sentinel-2 imagery
7. **Assess accuracy** using confusion matrices and statistical metrics
8. **Generate area statistics** for each land cover class
9. **Export results** for further analysis and visualization
10. **Apply classification** to real-world Natural Resource Management (NRM) challenges in the Philippines

---

## Study Area: Palawan Province

**Why Palawan?**

- **UNESCO Biosphere Reserve**: Home to exceptional biodiversity and endemic species
- **Conservation Priority**: Contains the Puerto Princesa Subterranean River National Park (UNESCO World Heritage Site)
- **Environmental Challenges**: Deforestation, mining, agricultural expansion, tourism impacts
- **NRM Relevance**: Critical for monitoring forest cover, mangroves, agricultural land conversion
- **Policy Context**: Palawan Strategic Environmental Plan (SEP) requires regular land cover monitoring

Palawan represents a critical case study where accurate land cover classification directly supports conservation planning and sustainable development decisions.

---

## Notebook Overview

This notebook guides participants through a complete supervised classification workflow using Random Forest:

```
Data Acquisition → Preprocessing → Feature Engineering → Training Data → Model Training → Classification → Validation → Export
```

**Key Concepts Covered:**
- Cloud computing for Earth Observation (Google Earth Engine)
- Spectral signatures and feature extraction
- Supervised machine learning (Random Forest)
- Model explainability (feature importance)
- Accuracy assessment and validation

---

## A. Setup and Authentication (10 minutes)

### 🎓 Teaching Notes:
- Walk through authentication process step-by-step
- Have participants raise hands when successfully authenticated
- Common issue: Browser blocking popups - suggest allowing them
- Backup: Pre-authenticated notebook if time is constrained

In [ ]:
# Install required packages (run only once)
!pip install earthengine-api geemap pandas numpy matplotlib seaborn -q

In [ ]:
# Import libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")
print(f"Earth Engine API version: {ee.__version__}")
print(f"Geemap version: {geemap.__version__}")

In [ ]:
# Authenticate and initialize Earth Engine
try:
    ee.Initialize()
    print("✓ Earth Engine initialized successfully")
except Exception as e:
    print("First-time authentication required...")
    ee.Authenticate()
    ee.Initialize()
    print("✓ Earth Engine authenticated and initialized")

In [ ]:
# Test connection with a simple query
test_image = ee.Image('COPERNICUS/S2_SR/20240115T015701_20240115T020226_T51PUS')
image_info = test_image.getInfo()

print("✓ Connection successful!")
print(f"Test Image ID: {image_info['id']}")
print(f"Available bands: {len(image_info['bands'])} bands")

In [ ]:
# Create interactive map centered on Palawan
# Palawan center coordinates: approximately 10.5°N, 118.8°E
Map = geemap.Map(center=[10.5, 118.8], zoom=8, height='600px')

# Add basemap options
Map.add_basemap('SATELLITE')

print("✓ Interactive map created")
print("Use the map controls to pan, zoom, and explore Palawan")
Map

---

## B. Study Area Definition (5 minutes)

### 🎓 Teaching Notes:
- Explain FAO GAUL dataset (global administrative boundaries)
- Note: Palawan may appear subdivided in recent data (Palawan province was split)
- Backup: Manual polygon if GAUL filter doesn't work
- Point out that study area definition significantly impacts computational cost

In [ ]:
# Define Palawan boundary using FAO GAUL dataset
philippines = ee.FeatureCollection('FAO/GAUL/2015/level1')

# Filter for Palawan province
palawan = philippines.filter(ee.Filter.eq('ADM1_NAME', 'Palawan'))

# Fallback: Manual boundary if needed
# palawan_coords = [
#     [117.5, 9.5], [117.5, 12.0], [119.5, 12.0], [119.5, 9.5], [117.5, 9.5]
# ]
# palawan = ee.Geometry.Polygon(palawan_coords)

print("✓ Palawan boundary defined")

In [ ]:
# Visualize the boundary on the map
Map.addLayer(palawan, {'color': 'red'}, 'Palawan Boundary')
Map.centerObject(palawan, 8)

# Display basic information
area_km2 = palawan.geometry().area().divide(1e6).getInfo()
bounds = palawan.geometry().bounds().getInfo()

print(f"Study Area: Palawan Province")
print(f"Approximate Area: {area_km2:,.0f} km²")
print(f"Bounding Box: {bounds['coordinates']}")

Map

---

## C. Sentinel-2 Data Acquisition (20 minutes)

### 🎓 Teaching Notes:
- Emphasize S2_SR (Level-2A) vs S2 (Level-1C) difference
- Discuss cloud cover threshold trade-off (lower = fewer images, higher = more cloud artifacts)
- Explain bitwise operations in cloud masking (many participants find this confusing)
- Show why median is preferred over mean for tropical regions
- Demo: Toggle between masked/unmasked to show cloud removal effectiveness

In [ ]:
# Define date range for imagery
start_date = '2024-01-01'
end_date = '2024-12-31'

print(f"Date range: {start_date} to {end_date}")

In [ ]:
# Load Sentinel-2 Surface Reflectance collection
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                 .filterBounds(palawan)
                 .filterDate(start_date, end_date)
                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))

collection_size = s2_collection.size().getInfo()
print(f"✓ Sentinel-2 collection loaded")
print(f"Number of images: {collection_size}")

if collection_size == 0:
    print("⚠️ Warning: No images found. Trying relaxed filter...")
    # Fallback with relaxed filter
    s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                     .filterBounds(palawan)
                     .filterDate(start_date, end_date)
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)))
    collection_size = s2_collection.size().getInfo()
    print(f"Relaxed filter: {collection_size} images found")

In [ ]:
# Define cloud masking function
def mask_s2_clouds(image):
    """
    Masks clouds and cirrus from Sentinel-2 imagery using the QA60 band.
    """
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus
    cloud_bit_mask = 1 << 10  # Binary: 10000000000 (bit 10)
    cirrus_bit_mask = 1 << 11  # Binary: 100000000000 (bit 11)
    
    # Both flags should be zero (clear conditions)
    mask = (qa.bitwiseAnd(cloud_bit_mask).eq(0)
            .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0)))
    
    # Return masked image, scaled to reflectance [0, 1]
    return image.updateMask(mask).divide(10000)

print("✓ Cloud masking function defined")

In [ ]:
# Apply cloud masking to the collection
s2_masked = s2_collection.map(mask_s2_clouds)

print("✓ Cloud masking applied to collection")

In [ ]:
# Create median composite
s2_composite = s2_masked.median().clip(palawan)

# Select bands for analysis
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
s2_composite = s2_composite.select(bands)

print("✓ Median composite created")
print(f"Selected bands: {bands}")

In [ ]:
# Visualize RGB composite
rgb_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.4
}

Map.addLayer(s2_composite, rgb_vis, 'Sentinel-2 RGB (True Color)')

print("✓ RGB composite added to map")
Map

In [ ]:
# Visualize false color composite (NIR-Red-Green)
false_color_vis = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0.0,
    'max': 0.4,
    'gamma': 1.2
}

Map.addLayer(s2_composite, false_color_vis, 'Sentinel-2 False Color (NIR-R-G)')

print("✓ False color composite added to map")
print("Red areas = healthy vegetation")
print("Green/brown areas = bare soil, urban")
print("Dark blue/black areas = water")

Map

### 🎓 Discussion Point: Composite Types

**Instructor Demo**: Show mean vs median composites side-by-side

**Expected observations:**
- Median: Sharper, fewer artifacts, better for classification
- Mean: Smoother, may retain cloud shadows
- Percentile: Useful for specific applications (e.g., 10th percentile for dark features)

In [ ]:
# INSTRUCTOR SOLUTION: Exercise 1 - Different composites

# Mean composite
s2_mean = s2_masked.mean().clip(palawan).select(bands)

# 25th percentile
s2_p25 = s2_masked.reduce(ee.Reducer.percentile([25])).clip(palawan)

# Visualize comparison
Map_composites = geemap.Map(center=[10.5, 118.8], zoom=9, height='600px')
Map_composites.addLayer(s2_composite, rgb_vis, 'Median (Best for classification)')
Map_composites.addLayer(s2_mean, rgb_vis, 'Mean', False)
Map_composites.addLayer(s2_p25, {'bands': ['B4_p25', 'B3_p25', 'B2_p25'], 'min': 0, 'max': 0.3}, '25th Percentile', False)
Map_composites.add_layer_control()

print("✓ Composite comparison ready")
print("Toggle layers to compare different compositing methods")

# Map_composites  # Uncomment to display

---

## D. Spectral Indices Calculation (15 minutes)

### 🎓 Teaching Notes:
- Use physical analogy: "Spectral indices are like medical tests - each reveals specific information"
- Draw connection to plant physiology (chlorophyll absorption, leaf structure)
- Show real examples on map: point out forested areas (high NDVI), water bodies (high NDWI), cities (high NDBI)
- Emphasize normalized indices are scale-invariant (work across sensors, dates)
- Common question: "Why normalize?" Answer: Reduces illumination effects, makes values comparable

In [ ]:
# Calculate NDVI
ndvi = s2_composite.normalizedDifference(['B8', 'B4']).rename('NDVI')

ndvi_vis = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['blue', 'white', 'green', 'darkgreen']
}

Map.addLayer(ndvi, ndvi_vis, 'NDVI')
print("✓ NDVI calculated and visualized")

In [ ]:
# Calculate NDWI
ndwi = s2_composite.normalizedDifference(['B3', 'B8']).rename('NDWI')

ndwi_vis = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['brown', 'white', 'lightblue', 'darkblue']
}

Map.addLayer(ndwi, ndwi_vis, 'NDWI')
print("✓ NDWI calculated and visualized")

In [ ]:
# Calculate NDBI
ndbi = s2_composite.normalizedDifference(['B11', 'B8']).rename('NDBI')

ndbi_vis = {
    'min': -0.3,
    'max': 0.3,
    'palette': ['green', 'white', 'red', 'darkred']
}

Map.addLayer(ndbi, ndbi_vis, 'NDBI')
print("✓ NDBI calculated and visualized")

In [ ]:
# Calculate EVI
evi = s2_composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': s2_composite.select('B8'),
        'RED': s2_composite.select('B4'),
        'BLUE': s2_composite.select('B2')
    }
).rename('EVI')

evi_vis = {
    'min': -0.2,
    'max': 0.8,
    'palette': ['blue', 'white', 'lightgreen', 'darkgreen']
}

Map.addLayer(evi, evi_vis, 'EVI')
print("✓ EVI calculated and visualized")

In [ ]:
# Side-by-side comparison
Map_indices = geemap.Map(center=[10.5, 118.8], zoom=9, height='600px')

Map_indices.addLayer(ndvi, ndvi_vis, 'NDVI')
Map_indices.addLayer(ndwi, ndwi_vis, 'NDWI')
Map_indices.addLayer(ndbi, ndbi_vis, 'NDBI')
Map_indices.addLayer(evi, evi_vis, 'EVI')
Map_indices.add_layer_control()

print("✓ All spectral indices visualized")
Map_indices

### 🎓 Discussion: Index Interpretation

**Ask participants:**
1. "Where do you see highest NDVI?" → Expected: Mountain forests
2. "Where is NDWI most positive?" → Expected: Rivers, bays, lakes
3. "Which index best separates forest from agriculture?" → Expected: NDVI, EVI

**Teaching point**: Spectral indices transform data into more discriminative feature space

In [ ]:
# INSTRUCTOR SOLUTION: Exercise 2 - Additional Indices

# SAVI (Soil-Adjusted Vegetation Index)
L = 0.5  # Soil brightness correction factor
savi = s2_composite.expression(
    '((NIR - RED) * (1 + L)) / (NIR + RED + L)',
    {
        'NIR': s2_composite.select('B8'),
        'RED': s2_composite.select('B4'),
        'L': L
    }
).rename('SAVI')

# GNDVI (Green NDVI)
gndvi = s2_composite.normalizedDifference(['B8', 'B3']).rename('GNDVI')

print("✓ SAVI and GNDVI calculated")
print("SAVI is useful for sparse vegetation (reduces soil background effect)")
print("GNDVI is sensitive to chlorophyll concentration")

---

## E. Feature Stack Preparation (10 minutes)

### 🎓 Teaching Notes:
- Explain: "Feature stack = multi-band image where each band is a feature"
- Analogy: "Like a medical patient record with multiple test results"
- Discuss curse of dimensionality briefly (too many features can hurt performance)
- Random Forest handles high dimensionality well compared to other algorithms

In [ ]:
# Create comprehensive feature stack
feature_stack = (s2_composite
                 .addBands(ndvi)
                 .addBands(ndwi)
                 .addBands(ndbi)
                 .addBands(evi))

# List all feature names
feature_names = feature_stack.bandNames().getInfo()

print("✓ Feature stack created")
print(f"Total features: {len(feature_names)}")
print(f"Feature list: {feature_names}")
print("\nFeature categories:")
print(f"  - Spectral bands: 6 (B2, B3, B4, B8, B11, B12)")
print(f"  - Spectral indices: 4 (NDVI, NDWI, NDBI, EVI)")
print(f"  - Total: {len(feature_names)} features")

In [ ]:
# Verify feature completeness
sample_point = ee.Geometry.Point([118.8, 10.5])
sample_values = feature_stack.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=sample_point,
    scale=10
).getInfo()

print("Sample feature values at test point (Central Palawan):")
for feature, value in sample_values.items():
    print(f"  {feature}: {value:.4f}" if value is not None else f"  {feature}: NULL")

null_features = [f for f, v in sample_values.items() if v is None]
if null_features:
    print(f"⚠️ Warning: Null values found in {null_features}")
else:
    print("\n✓ All features have valid values")

---

## F. Training Data Preparation (20 minutes)

### 🎓 Teaching Notes:
- **CRITICAL SECTION** - Stress that training data quality determines success
- Show examples of good vs bad training samples on map
- Discuss common errors:
  - Mixed pixels (polygon overlaps multiple classes)
  - Seasonal mismatch (training data from different time than imagery)
  - Class definition ambiguity (what counts as "forest" vs "agriculture"?)
- Mention data-centric AI: Improving data > improving model
- Interactive activity: Have participants suggest where to place training samples

In [ ]:
# Define class properties
class_info = {
    1: {'name': 'Forest', 'color': '006400'},
    2: {'name': 'Agriculture', 'color': 'FFFF00'},
    3: {'name': 'Water', 'color': '0000FF'},
    4: {'name': 'Urban', 'color': 'FF0000'},
    5: {'name': 'Bare Soil', 'color': '8B4513'}
}

print("Land Cover Classification Scheme:")
for class_id, info in class_info.items():
    print(f"  {class_id}: {info['name']}")

In [ ]:
# Create pre-defined training samples
# These coordinates are carefully selected based on visual interpretation
# In operational workflow, these would come from field surveys or expert digitization

# Forest samples (mountainous interior, dense vegetation)
forest_coords = [
    [[118.5, 10.5], [118.5, 10.52], [118.52, 10.52], [118.52, 10.5]],
    [[118.7, 10.8], [118.7, 10.82], [118.72, 10.82], [118.72, 10.8]],
    [[119.0, 11.0], [119.0, 11.02], [119.02, 11.02], [119.02, 11.0]],
    [[118.3, 9.8], [118.3, 9.82], [118.32, 9.82], [118.32, 9.8]],
    [[118.85, 10.3], [118.85, 10.32], [118.87, 10.32], [118.87, 10.3]],
]

# Agriculture samples (coastal plains, river valleys)
agriculture_coords = [
    [[118.7, 9.75], [118.7, 9.77], [118.72, 9.77], [118.72, 9.75]],
    [[118.9, 10.2], [118.9, 10.22], [118.92, 10.22], [118.92, 10.2]],
    [[118.55, 10.0], [118.55, 10.02], [118.57, 10.02], [118.57, 10.0]],
    [[119.15, 10.9], [119.15, 10.92], [119.17, 10.92], [119.17, 10.9]],
]

# Water samples (bays, rivers, lakes)
water_coords = [
    [[118.73, 9.73], [118.73, 9.75], [118.75, 9.75], [118.75, 9.73]],  # Puerto Princesa Bay
    [[119.3, 10.5], [119.3, 10.52], [119.32, 10.52], [119.32, 10.5]],  # Coastal water
    [[118.4, 10.3], [118.4, 10.32], [118.42, 10.32], [118.42, 10.3]],
    [[118.95, 9.6], [118.95, 9.62], [118.97, 9.62], [118.97, 9.6]],
]

# Urban samples (Puerto Princesa, towns)
urban_coords = [
    [[118.74, 9.74], [118.74, 9.76], [118.76, 9.76], [118.76, 9.74]],  # Puerto Princesa City
    [[119.08, 10.82], [119.08, 10.84], [119.10, 10.84], [119.10, 10.82]],  # Taytay
    [[118.77, 9.77], [118.77, 9.79], [118.79, 9.79], [118.79, 9.77]],
]

# Bare soil samples (mining, cleared land)
bare_coords = [
    [[117.95, 9.4], [117.95, 9.42], [117.97, 9.42], [117.97, 9.4]],
    [[118.2, 10.1], [118.2, 10.12], [118.22, 10.12], [118.22, 10.1]],
    [[118.6, 10.6], [118.6, 10.62], [118.62, 10.62], [118.62, 10.6]],
]

# Convert to ee.FeatureCollection
def create_training_features(coords_list, class_value):
    """Convert coordinate list to ee.FeatureCollection with class label."""
    features = []
    for coords in coords_list:
        polygon = ee.Geometry.Polygon(coords)
        feature = ee.Feature(polygon, {'landcover': class_value})
        features.append(feature)
    return ee.FeatureCollection(features)

forest_fc = create_training_features(forest_coords, 1)
agriculture_fc = create_training_features(agriculture_coords, 2)
water_fc = create_training_features(water_coords, 3)
urban_fc = create_training_features(urban_coords, 4)
bare_fc = create_training_features(bare_coords, 5)

# Merge all training samples
training_polygons = (forest_fc
                     .merge(agriculture_fc)
                     .merge(water_fc)
                     .merge(urban_fc)
                     .merge(bare_fc))

print("✓ Training polygons created")
print(f"Total training polygons: {training_polygons.size().getInfo()}")

In [ ]:
# Visualize training polygons
Map_training = geemap.Map(center=[10.5, 118.8], zoom=8, height='600px')

Map_training.addLayer(s2_composite, rgb_vis, 'Sentinel-2 RGB')

for class_id, info in class_info.items():
    class_polygons = training_polygons.filter(ee.Filter.eq('landcover', class_id))
    Map_training.addLayer(
        class_polygons,
        {'color': info['color']},
        f"Training: {info['name']}"
    )

print("✓ Training polygons visualized")
Map_training

In [ ]:
# Sample training data from feature stack
training_samples = feature_stack.sampleRegions(
    collection=training_polygons,
    properties=['landcover'],
    scale=10,
    geometries=False,
    tileScale=4
)

sample_count = training_samples.size().getInfo()
print(f"✓ Training samples extracted")
print(f"Total training pixels: {sample_count}")

In [ ]:
# Inspect sample distribution by class
class_counts = {}
for class_id in range(1, 6):
    class_samples = training_samples.filter(ee.Filter.eq('landcover', class_id))
    count = class_samples.size().getInfo()
    class_counts[class_id] = count

print("\nSample distribution by class:")
for class_id, count in class_counts.items():
    class_name = class_info[class_id]['name']
    print(f"  {class_name}: {count} pixels")

max_count = max(class_counts.values())
min_count = min(class_counts.values())
imbalance_ratio = max_count / min_count if min_count > 0 else float('inf')

if imbalance_ratio > 3:
    print(f"\n⚠️ Warning: Class imbalance detected (ratio: {imbalance_ratio:.1f}:1)")
    print("Consider adding more samples for underrepresented classes")
else:
    print(f"\n✓ Class distribution is balanced (ratio: {imbalance_ratio:.1f}:1)")

### 🎓 Discussion: Training Data Quality

**Ask participants:**
- "Is this enough training data?" → Context-dependent, but more is generally better
- "How would you collect training data in the field?" → GPS surveys, local knowledge
- "What if classes are imbalanced?" → Options: collect more samples, resampling, class weighting

In [ ]:
# Explore training data
sample_limit = training_samples.limit(100)
sample_list = sample_limit.getInfo()['features']

sample_data = [feature['properties'] for feature in sample_list]
df_samples = pd.DataFrame(sample_data)

print("Sample training data (first 10 rows):")
print(df_samples.head(10))

print("\nFeature statistics by class:")
print(df_samples.groupby('landcover')[['NDVI', 'NDWI', 'B4', 'B8']].mean())

---

## G. Random Forest Training (20 minutes)

### 🎓 Teaching Notes:
- Draw Random Forest diagram on whiteboard/slide: multiple trees → majority vote
- Explain bagging (bootstrap aggregating): each tree sees random subset
- Explain feature randomness: each split considers random subset of features
- Discuss hyperparameters:
  - **numberOfTrees**: More = better (diminishing returns after ~100-200)
  - **variablesPerSplit**: Auto = sqrt(n) is theoretically optimal
  - **minLeafPopulation**: Controls tree depth (overfitting prevention)
- Mention: RF is embarrassingly parallel (scales well)

In [ ]:
# Configure Random Forest classifier
rf_classifier = ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=None,  # Auto: sqrt(10) ≈ 3 features per split
    minLeafPopulation=1,
    bagFraction=0.632,  # Out-of-bag fraction
    seed=42
)

print("✓ Random Forest classifier configured")
print("Configuration:")
print(f"  - Number of trees: 100")
print(f"  - Variables per split: auto (~{int(np.sqrt(len(feature_names)))} features)")
print(f"  - Min leaf population: 1")
print(f"  - Random seed: 42 (for reproducibility)")

In [ ]:
# Train the classifier
print("Training Random Forest classifier...")
print("This may take 1-2 minutes...")

trained_classifier = rf_classifier.train(
    features=training_samples,
    classProperty='landcover',
    inputProperties=feature_names
)

print("✓ Random Forest training complete")

In [ ]:
# Extract feature importance (if available)
try:
    importance = trained_classifier.explain().get('importance')
    importance_dict = importance.getInfo()
    
    df_importance = pd.DataFrame({
        'Feature': list(importance_dict.keys()),
        'Importance': list(importance_dict.values())
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.barh(df_importance['Feature'], df_importance['Importance'])
    plt.xlabel('Importance', fontsize=12)
    plt.ylabel('Feature', fontsize=12)
    plt.title('Random Forest Feature Importance', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\nTop 5 most important features:")
    print(df_importance.head(5).to_string(index=False))
    
except Exception as e:
    print("⚠️ Feature importance not available from GEE smileRandomForest")
    print("\n🎓 INSTRUCTOR NOTE: This is expected - GEE's SMILE implementation doesn't expose importance")
    print("\nExpected important features (based on EO literature):")
    print("  1. NIR band (B8) - vegetation discrimination")
    print("  2. NDVI - vegetation index")
    print("  3. SWIR bands (B11, B12) - moisture, urban")
    print("  4. Red band (B4) - vegetation contrast")
    print("  5. NDWI - water detection")

### 🎓 Discussion: Why Random Forest?

**Ask participants:** "What are advantages of Random Forest over a single decision tree?"

**Expected answers:**
- More stable (less sensitive to training data)
- Less prone to overfitting
- Better generalization
- Provides feature importance

**Follow-up:** "When might a single tree be better?"
- When interpretability is critical (single tree is easier to visualize)
- When computational resources are very limited

In [ ]:
# INSTRUCTOR SOLUTION: Exercise 4 - Different tree counts

# Train with 50 trees
rf_50 = ee.Classifier.smileRandomForest(numberOfTrees=50, seed=42)
trained_rf_50 = rf_50.train(
    features=training_samples,
    classProperty='landcover',
    inputProperties=feature_names
)

# Train with 200 trees
rf_200 = ee.Classifier.smileRandomForest(numberOfTrees=200, seed=42)
trained_rf_200 = rf_200.train(
    features=training_samples,
    classProperty='landcover',
    inputProperties=feature_names
)

print("✓ Trained RF with 50 trees")
print("✓ Trained RF with 200 trees")
print("\n🎓 INSTRUCTOR NOTE: Performance difference is usually <5% beyond 100 trees")
print("Computational cost increases linearly with tree count")

---

## H. Image Classification (15 minutes)

### 🎓 Teaching Notes:
- Explain: Classification applies trained model to every pixel
- Show before/after: RGB → classified image
- Interactive activity: Have participants zoom to different areas and assess visually
- Point out common patterns:
  - Forest in mountains
  - Agriculture in plains
  - Water clearly delineated
  - Urban concentrated in Puerto Princesa
- Discuss visual artifacts (isolated pixels, boundary effects)

In [ ]:
# Apply classifier to feature stack
print("Classifying image...")
print("This may take 1-2 minutes...")

classified_image = feature_stack.classify(trained_classifier)

print("✓ Classification complete")

In [ ]:
# Define color palette
class_palette = [class_info[i]['color'] for i in sorted(class_info.keys())]
print(f"Classification palette: {class_palette}")

In [ ]:
# Visualize classification result
Map_classified = geemap.Map(center=[10.5, 118.8], zoom=8, height='700px')

Map_classified.addLayer(s2_composite, rgb_vis, 'Sentinel-2 RGB', False)
Map_classified.addLayer(s2_composite, false_color_vis, 'False Color', False)

Map_classified.addLayer(
    classified_image,
    {'min': 1, 'max': 5, 'palette': class_palette},
    'Land Cover Classification'
)

for class_id, info in class_info.items():
    class_polygons = training_polygons.filter(ee.Filter.eq('landcover', class_id))
    Map_classified.addLayer(
        class_polygons,
        {'color': info['color']},
        f"Training: {info['name']}",
        False
    )

Map_classified.add_layer_control()

print("✓ Classification visualized")
print("\nLegend:")
for class_id, info in class_info.items():
    print(f"  {info['name']}: #{info['color']}")

Map_classified

### 🎓 Interactive Activity: Visual Assessment

**Instructor-led exploration (5-10 minutes):**

1. **Puerto Princesa City** (9.74°N, 118.74°E)
   - Toggle RGB and classification
   - Is urban area correctly identified?
   - Any confusion with bare soil?

2. **Mount Mantalingahan** (9°N, 118.5°E) - highest peak
   - Primary forest should dominate
   - Check for misclassification at cloud shadows

3. **Coastal mangroves** (10.3°N, 119.3°E)
   - Should be classified as forest or water?
   - Discuss spectral similarity to terrestrial forest

4. **Agricultural plains** (10°N, 118.7°E)
   - Rice paddies vs. bare soil confusion
   - Seasonal effects (planting vs. harvest)

**Ask participants to note 3 misclassifications for discussion**

---

## I. Accuracy Assessment (25 minutes)

### 🎓 Teaching Notes:
- **CRITICAL SECTION** - Participants often skip proper validation
- Emphasize: Visual inspection ≠ accuracy assessment
- Explain confusion matrix carefully:
  - Draw on board: rows = reference, columns = predicted
  - Diagonal = correct, off-diagonal = errors
- Discuss metrics:
  - Overall accuracy: Simple but can be misleading with imbalance
  - Producer's accuracy: "How complete is the map?" (recall)
  - User's accuracy: "How reliable is the map?" (precision)
  - Kappa: Accounts for chance agreement
- Common question: "What accuracy is good enough?" → Context-dependent, but >80% overall is typical target

In [ ]:
# Train-validation split
training_samples = training_samples.randomColumn('random', seed=42)

split = 0.8
training_set = training_samples.filter(ee.Filter.lt('random', split))
validation_set = training_samples.filter(ee.Filter.gte('random', split))

train_count = training_set.size().getInfo()
val_count = validation_set.size().getInfo()

print("Data split:")
print(f"  Training samples: {train_count} ({train_count/(train_count+val_count)*100:.1f}%)")
print(f"  Validation samples: {val_count} ({val_count/(train_count+val_count)*100:.1f}%)")

In [ ]:
# Retrain on training set only
print("Retraining classifier on training set...")

rf_final = ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=None,
    minLeafPopulation=1,
    seed=42
)

trained_final = rf_final.train(
    features=training_set,
    classProperty='landcover',
    inputProperties=feature_names
)

print("✓ Retraining complete")

In [ ]:
# Classify validation set
validation_classified = validation_set.classify(trained_final)
print("✓ Validation set classified")

In [ ]:
# Generate confusion matrix
confusion_matrix = validation_classified.errorMatrix('landcover', 'classification')
matrix_array = confusion_matrix.array().getInfo()

print("Confusion Matrix:")
print("Rows = Reference (True), Columns = Predicted")
print(matrix_array)
print("\n🎓 INSTRUCTOR NOTE: Walk through matrix interpretation with participants")

In [ ]:
# Visualize confusion matrix as heatmap
class_names = [class_info[i]['name'] for i in sorted(class_info.keys())]

plt.figure(figsize=(10, 8))
sns.heatmap(
    matrix_array,
    annot=True,
    fmt='g',
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names,
    cbar_kws={'label': 'Count'}
)
plt.xlabel('Predicted Class', fontsize=12)
plt.ylabel('Reference Class', fontsize=12)
plt.title('Confusion Matrix - Palawan Land Cover Classification', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n🎓 INSTRUCTOR PROMPT: Ask participants to identify:")
print("1. Which classes have highest accuracy? (look at diagonal)")
print("2. Which classes are most confused? (look at off-diagonal)")
print("3. Are there systematic errors? (e.g., consistent confusion between two classes)")

In [ ]:
# Calculate accuracy metrics
overall_accuracy = confusion_matrix.accuracy().getInfo()
print(f"Overall Accuracy: {overall_accuracy*100:.2f}%")

kappa = confusion_matrix.kappa().getInfo()
print(f"Kappa Coefficient: {kappa:.4f}")

if kappa > 0.8:
    kappa_interp = "Excellent agreement"
elif kappa > 0.6:
    kappa_interp = "Substantial agreement"
elif kappa > 0.4:
    kappa_interp = "Moderate agreement"
else:
    kappa_interp = "Poor agreement"

print(f"Interpretation: {kappa_interp}")

print("\n🎓 INSTRUCTOR NOTE:")
if overall_accuracy > 0.85:
    print("Excellent accuracy! Likely due to well-chosen training samples.")
elif overall_accuracy > 0.7:
    print("Good accuracy. Typical for initial classification attempt.")
else:
    print("Low accuracy. Discuss potential causes:")
    print("  - Insufficient training samples")
    print("  - Poor quality training data")
    print("  - Class confusion (need to refine definitions)")
    print("  - Need more discriminative features")

In [ ]:
# Per-class accuracy
producers_accuracy = confusion_matrix.producersAccuracy().getInfo()
users_accuracy = confusion_matrix.consumersAccuracy().getInfo()

print("\nProducer's Accuracy (Recall):")
for i, acc in enumerate(producers_accuracy):
    class_name = class_info[i+1]['name']
    print(f"  {class_name}: {acc*100:.2f}%")

print("\nUser's Accuracy (Precision):")
for i, acc in enumerate(users_accuracy):
    class_name = class_info[i+1]['name']
    print(f"  {class_name}: {acc*100:.2f}%")

In [ ]:
# Summary DataFrame
df_accuracy = pd.DataFrame({
    'Class': class_names,
    'Producer\'s Accuracy (%)': [acc*100 for acc in producers_accuracy],
    'User\'s Accuracy (%)': [acc*100 for acc in users_accuracy]
})

print("\nAccuracy Summary by Class:")
print(df_accuracy.to_string(index=False))

In [ ]:
# Visualize per-class accuracy
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(class_names))
width = 0.35

ax.bar(x - width/2, df_accuracy['Producer\'s Accuracy (%)'], width, label='Producer\'s Accuracy', alpha=0.8)
ax.bar(x + width/2, df_accuracy['User\'s Accuracy (%)'], width, label='User\'s Accuracy', alpha=0.8)

ax.set_xlabel('Land Cover Class', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Per-Class Accuracy Metrics', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(class_names)
ax.legend()
ax.set_ylim([0, 105])
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🎓 INSTRUCTOR DISCUSSION PROMPT:")
print("Analyze accuracy differences:")
print("- High producer's, low user's = over-prediction (too many false positives)")
print("- Low producer's, high user's = under-prediction (too many false negatives)")
print("- Both low = poor discrimination from other classes")

### 🎓 Teaching Point: Error Analysis

**Common confusion patterns:**

1. **Agriculture ↔ Bare Soil**
   - Cause: Bare agricultural fields (post-harvest, pre-planting)
   - Solution: Multi-temporal data (capture full phenological cycle)

2. **Forest ↔ Agriculture**
   - Cause: High-biomass crops (sugarcane, perennial crops)
   - Solution: Add texture features, time series

3. **Urban ↔ Bare Soil**
   - Cause: Similar spectral response (concrete vs. soil)
   - Solution: Texture, spatial context, NDBI weighting

4. **Water ↔ Shadow**
   - Cause: Cloud shadows have low reflectance
   - Solution: Better cloud masking, topographic correction

In [ ]:
# INSTRUCTOR SOLUTION: Exercise 6 - Different splits

print("🎓 INSTRUCTOR DEMO: Testing 70-30 split\n")

split_70 = 0.7
training_set_70 = training_samples.filter(ee.Filter.lt('random', split_70))
validation_set_70 = training_samples.filter(ee.Filter.gte('random', split_70))

# Retrain
rf_70 = ee.Classifier.smileRandomForest(numberOfTrees=100, seed=42)
trained_70 = rf_70.train(
    features=training_set_70,
    classProperty='landcover',
    inputProperties=feature_names
)

# Evaluate
validation_classified_70 = validation_set_70.classify(trained_70)
confusion_70 = validation_classified_70.errorMatrix('landcover', 'classification')
accuracy_70 = confusion_70.accuracy().getInfo()

print(f"80-20 split accuracy: {overall_accuracy*100:.2f}%")
print(f"70-30 split accuracy: {accuracy_70*100:.2f}%")
print("\nExpected outcome: Similar accuracy, larger validation set gives more reliable estimate")

---

## J. Area Statistics (10 minutes)

### 🎓 Teaching Notes:
- Connect to policy/management: "These numbers inform decision-making"
- Discuss uncertainty: "Accuracy affects area estimates - propagate error!"
- Compare with official statistics if available
- Philippine context: DENR requires regular forest cover reporting

In [ ]:
# Calculate area per class
pixel_area = ee.Image.pixelArea()
area_image = pixel_area.addBands(classified_image)

area_stats = area_image.reduceRegion(
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='landcover'
    ),
    geometry=palawan,
    scale=10,
    maxPixels=1e10,
    tileScale=4
)

print("Calculating area statistics...")
print("This may take 2-3 minutes...")

In [ ]:
# Extract and process results
area_results = area_stats.getInfo()
area_groups = area_results['groups']

area_data = []
for group in area_groups:
    class_id = group['landcover']
    area_m2 = group['sum']
    area_km2 = area_m2 / 1e6
    area_ha = area_m2 / 1e4
    
    area_data.append({
        'Class ID': class_id,
        'Class Name': class_info[class_id]['name'],
        'Area (km²)': area_km2,
        'Area (ha)': area_ha
    })

df_area = pd.DataFrame(area_data).sort_values('Area (km²)', ascending=False)

total_area = df_area['Area (km²)'].sum()
df_area['Percentage (%)'] = (df_area['Area (km²)'] / total_area) * 100

print("\nLand Cover Area Statistics:")
print(df_area.to_string(index=False))
print(f"\nTotal classified area: {total_area:,.0f} km²")
print(f"Palawan total area: ~14,649 km²")

In [ ]:
# Visualize area distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

colors = [f"#{class_info[row['Class ID']]['color']}" for _, row in df_area.iterrows()]

# Bar chart
ax1.bar(df_area['Class Name'], df_area['Area (km²)'], color=colors, alpha=0.7)
ax1.set_xlabel('Land Cover Class', fontsize=12)
ax1.set_ylabel('Area (km²)', fontsize=12)
ax1.set_title('Land Cover Distribution', fontsize=14, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)

# Pie chart
ax2.pie(
    df_area['Percentage (%)'],
    labels=df_area['Class Name'],
    autopct='%1.1f%%',
    colors=colors,
    startangle=90
)
ax2.set_title('Land Cover Percentage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n🎓 INSTRUCTOR DISCUSSION:")
print("Compare results with expected patterns:")
print("- Palawan is ~50-60% forested (DENR estimates)")
print("- Agriculture mainly coastal plains (<15%)")
print("- Urban very small (<2%)")
print("\nDiscuss potential discrepancies and their causes")

---

## K. Export Results (10 minutes)

### 🎓 Teaching Notes:
- Quick section - focus on export parameters
- Emphasize: Exports run asynchronously (don't block)
- Common issues: Memory limits, quota exceeded, file size
- Show task manager: https://code.earthengine.google.com/tasks

In [ ]:
# Export classification
export_classification = ee.batch.Export.image.toDrive(
    image=classified_image,
    description='Palawan_LandCover_Classification',
    folder='EarthEngine',
    fileNamePrefix='palawan_landcover_2024',
    region=palawan.geometry(),
    scale=10,
    crs='EPSG:4326',
    maxPixels=1e10,
    fileFormat='GeoTIFF'
)

export_classification.start()

print("✓ Classification export task started")
print(f"Task: {export_classification.status()['description']}")
print(f"State: {export_classification.status()['state']}")

In [ ]:
# Export training samples
export_training = ee.batch.Export.table.toDrive(
    collection=training_samples,
    description='Palawan_Training_Samples',
    folder='EarthEngine',
    fileNamePrefix='palawan_training_samples',
    fileFormat='CSV'
)

export_training.start()
print("✓ Training samples export started")

In [ ]:
# Save local files
df_confusion = pd.DataFrame(matrix_array, index=class_names, columns=class_names)
df_confusion.to_csv('palawan_confusion_matrix.csv')

df_area.to_csv('palawan_area_statistics.csv', index=False)

print("✓ Local files saved")
print("  - palawan_confusion_matrix.csv")
print("  - palawan_area_statistics.csv")

---

## Summary and Wrap-up (5 minutes)

### 🎓 Closing Discussion

**Ask participants:**
1. "What was the most challenging part?" → Common: Training data creation
2. "What surprised you about the results?" → Varies
3. "How would you apply this in your work?" → Encourage specific applications

**Key takeaways to emphasize:**
- Training data quality is paramount
- Random Forest is powerful but not magic
- Accuracy assessment is non-negotiable
- GEE enables scalable analysis
- Operational deployment requires iteration

**Preview next sessions:**
- Tomorrow: Deep learning for more complex problems
- Day 3: Change detection, time series analysis
- Day 4: Operational deployment

---

## Additional Teaching Resources

### Backup Exercises (if time permits)

1. **Texture features**: Add GLCM (Gray-Level Co-occurrence Matrix) texture
2. **Topography**: Add elevation, slope, aspect from SRTM
3. **Multi-temporal**: Compare dry vs wet season
4. **Other algorithms**: Compare with SVM, CART
5. **Post-processing**: Majority filter, morphological operations

### Common Issues and Solutions

**Issue: "No images found"**
- Solution: Use fallback date range, relax cloud filter

**Issue: "Memory limit exceeded"**
- Solution: Add `tileScale=4` or `tileScale=8`

**Issue: "Very low accuracy"**
- Check training data quality visually
- Ensure class definitions are clear
- Add more training samples

**Issue: "Classification has many isolated pixels"**
- Normal for pixel-based classification
- Solution: Apply majority filter (post-processing)

### Assessment Questions

**Knowledge check:**
1. Why use median composite instead of mean?
2. What does NDVI measure physically?
3. What's the difference between producer's and user's accuracy?
4. Why split data into training and validation sets?
5. How does Random Forest prevent overfitting?

**Application:**
1. How would you adapt this for change detection?
2. What additional features would help separate agriculture from bare soil?
3. How would you validate results without field data?

---

**END OF INSTRUCTOR VERSION**